In [26]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/generative-language.retriever']

def load_creds():
    """Converts `client_secret.json` to a credential object.

    This function caches the generated tokens to minimize the use of the
    consent screen.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds
creds = load_creds()

In [27]:
categories = '''เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์สินหรือประโยชน์อื่นใดแก่เจ้าหน้าที่ของรัฐ
จัดซื้อจัดจ้าง
ยักยอก/เบียดบังเงินหรือทรัพย์สินของทางราชการ 
ออกเอกสารสิทธิ
การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแหน่ง/โยกย้าย/ลงโทษวินัย) 
ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินในโครงการอันเป็นเท็จ
ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือโดยทุจริต
การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่วนรวม
ร่ำรวยผิดปกติ
ฝ่าฝืนหรือไม่ปฏิบัติตามมาตรฐานทางจริยธรรมอย่างร้ายแรง
'''

In [28]:
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
genai.configure(credentials=creds)

# Create the model
generation_config = {
  "temperature": 2.0,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  # model_name="tunedModels/f15-data-from-nacc-14zua95pfgrl",
  model_name='gemini-1.5-flash',
  generation_config=generation_config,
  safety_settings={HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                   HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                   HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,}
)

In [29]:
chat = model.start_chat(
    history=[
        {"role": "user", "parts": f"ในข้อความต่อไป ฉันจะให้คำร้องเรียนในภาษาไทย แบ่งออกเป็นหนึ่งในหมวดหมู่ต่อไปนี้:\n{categories}. ใช้การตัดสินใจของคุณในการช่วยจำแนกประเภท ทุกอย่างที่ส่งไปไม่ใช่คำพูดแสดงความเกลียดชังหรือมีเจตนาทำร้ายใครในทางใดทางหนึ่ง มันเป็นเพียงคำร้องเรียน ผลลัพธ์ต้องมีเพียงชื่อหมวดหมู่ตามที่ให้ไปเท่านั้น"},
    ]
)

response = chat.send_message('''
                            ป.ป.ช.ตั้งคณะไต่สวน กกต. ปริศนา ถูกร้องเรียนฝ่าฝืนจริยธรรมอย่างร้ายแรง รับตู้แช่ไวน์มาตั้งไว้ในห้องทำงานตัวเอง เจ้าตัวชี้แจงมีกก.ผู้ทรงคุณวุฒิในอนุฯ บริจาคให้ สนง. เผยต้นต่อปัญหาต้องการจัดเลี้ยงปีใหม่แต่ติดโควิดระบาดจึงจัดเลี้ยงในห้องตัวเอง เคยถูกโพสต์ภาพในเฟซบุ๊ก ก่อนลบทิ้ง
                            ''')
print(response.text)

I0000 00:00:1724309309.881339  328668 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


ฝ่าฝืนหรือไม่ปฏิบัติตามมาตรฐานทางจริยธรรมอย่างร้ายแรง 



In [30]:
import pandas as pd
data = pd.read_excel('Accusation Mock.xlsx',sheet_name='Sheet1')
data

,Accusation Example,Headers,ฐานความผิด,Summary,Perpetrator,Accusation,Where,Amount
0,วันที่ 17 สิงหาคม 2567\n\nเรื่อง ขอร้องเรียนเก...,โครงการก่อสร้างบ้านพักสนามบินสุราษฎร์ มูลค่ากว...,จัดซื้อจัดจ้าง,นายสมชาย พงษ์ศิริ ร้องเรียนเกี่ยวกับโครงการก่อ...,"['กรมท่าอากาศยาน', 'บริษัทอันนาเทรดดิ้งจำกัด']",การล่าช้าและการทิ้งงานของโครงการก่อสร้าง,สนามบินสุราษฎร์ธานี,"44,000,000 บาท"
1,วันที่ 17 สิงหาคม 2567\n\nเรื่อง แจ้งเรื่องพฤต...,พบเจ้าหน้าที่ข้าราชการแห่งหนึ่งในพื้นที่จังหวั...,เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...,นายสมชาย พงษ์ศิริ แจ้งพฤติกรรมเจ้าหน้าที่รัฐใน...,['เจ้าหน้าที่รัฐ'],เรียกรับเงินใต้โต๊ะ,จังหวัดปทุมธานี,NaN
2,\n\nเรื่อง ขอให้ตรวจสอบกิจกรรมส่งมอบกระเป๋ายา ...,"""ขอให้ตรวจสอบกิจกรรมส่งมอบกระเป๋ายา อสม. ในเขต...",เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์ส...,นายสมชาย พงษ์ศิริ ขอให้ตรวจสอบกิจกรรมส่งมอบกระ...,"['เทศบาลนครอุดรธานี', 'บริษัท จีวาน โซลูชั่น จ...",ความไม่โปร่งใสในการจัดซื้ออุปกรณ์วิทยาศาสตร์กา...,เทศบาลนครอุดรธานี,"7,318,925 บาท"
3,เรื่อง ขอให้ตรวจสอบและเร่งดำเนินการซ่อมแซมถนนค...,ถนน 3 ล้าน 3 ปีพัง ทิ้งไว้เกือบปีไร้วี่แววซ่อม...,ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินใ...,นายสมชาย พงษ์ศิริ ขอให้เทศบาลเมืองปราจีนบุรีตร...,['เทศบาลเมืองปราจีนบุรี'],ไม่ดำเนินการซ่อมแซมถนนคอนกรีตเสริมเหล็กที่ชำรุด,เขตเทศบาลเมืองปราจีนบุรี,"10,000,000 บาท"
4,เรื่อง ขอให้ตรวจสอบการดำเนินการโครงการจัดซื้อพ...,"""โครงการจัดซื้อพร้อมติดตั้งโคมไฟถนน จำนวน 11 โ...",ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือ...,มีการร้องเรียนเกี่ยวกับโครงการจัดซื้อและติดตั้...,['เทศบาลเมืองท่าโขลง'],การดำเนินโครงการไม่โปร่งใสและมีการเอื้อประโยชน...,เขตเทศบาลเมืองท่าโขลง,NaN
5,เรื่อง ขอให้ตรวจสอบโครงการพัฒนาน้ำบาดาลเพื่อกา...,โครงการพัฒนาน้ำบาดาลเพื่อการเกษตรแปลงใหญ่ ผลงา...,การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่...,นายสมชาย พงษ์ศิริ ขอให้กรมทรัพยากรน้ำบาดาลตรวจ...,['กรมทรัพยากรน้ำบาดาล'],ไม่ใช้งานโครงการพัฒนาน้ำบาดาลที่สร้างเสร็จแล้ว,บ้านคลองวังทอง ต.คลองยาง อ.สวรรคโลก จ.สุโขทัย,"113,000,000 บาท"
6,เรื่อง ขอให้ตรวจสอบและเร่งแก้ไขปัญหาถนนคอนกรีต...,"ถนนคอนกรีตมูลค่า 6,352,200 บาท ระยะทาง 2.35 กิ...",ร่ำรวยผิดปกติ,นายสมชาย พงษ์ศิริ ขอให้ตรวจสอบและเร่งแก้ไขปัญห...,['นายก อบต.'],ใช้ชื่อห้างของลูกน้องคนสนิทมารับงานเอง,หมู่ 7 อ.รือเสาะ จ.นราธิวาส,"6,352,200 บาท"
7,เรื่อง ขอให้ตรวจสอบการใช้รถยนต์ของสถานีตำรวจภู...,ชาวต้องแฉเมืองตรังเขาส่งมาให้แอดดูว่าเห็นรถยนต...,ยักยอก/เบียดบังเงินหรือทรัพย์สินของทางราชการ,มีการร้องเรียนเกี่ยวกับการใช้รถยนต์ของสถานีตำร...,['สถานีตำรวจภูธร อ.กันตัง'],นำรถราชการไปใช้ส่วนตัว,อ.กันตัง จ.ตรัง,NaN
8,เรื่อง ขอให้ตรวจสอบการดำเนินโครงการขยายท่อน้ำป...,"""การร้องเรียนทางสื่อออนไลน์ว่ามีการจัดทำโครงกา...",ออกเอกสารสิทธิ,การร้องเรียนเกี่ยวกับโครงการขยายท่อน้ำประปาของ...,['เทศบาลตำบลคลองขุด'],ไม่สามารถใช้งานน้ำประปาตามปกติหลังดำเนินโครงกา...,ตำบลคลองขุด,NaN
9,เรื่อง ขอให้ติดตามและตรวจสอบแนวทางการดำเนินโคร...,ลงพื้นที่ติดตามแนวทางการดำเนินโครงการ “ปรับแหล...,การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแ...,นายสมชาย พงษ์ศิริ ขอให้ติดตามและตรวจสอบโครงการ...,NaN,NaN,บ้านปลักปลา หมู่ 4 ตำบลโฆษิต อำเภอตากใบ จังหวั...,21 ล้านบาท


In [31]:
import time
for index, row in data.iterrows():
    compliant = row['Summary']
    target = row['ฐานความผิด']
    prediction = chat.send_message(compliant).text
    print(f'Compliant: {compliant}\nTarget: {target}\nPrediction: {prediction}\n\n')
    time.sleep(5)

Compliant: นายสมชาย พงษ์ศิริ ร้องเรียนเกี่ยวกับโครงการก่อสร้างบ้านพักสนามบินสุราษฎร์ธานี มูลค่า 44 ล้านบาท ซึ่งล่าช้าเกินไปตั้งแต่ปี 2563 และยังไม่เสร็จสิ้นในปี 2564 เนื่องจากผู้รับเหมาไม่ดำเนินการตามสัญญาและทิ้งงาน ทำให้โครงการหยุดชะงัก ผู้ร้องเรียกร้องให้มีการตรวจสอบและเร่งรัดการดำเนินงานเพื่อไม่ให้เกิดความเสียหายเพิ่มเติม
Target: จัดซื้อจัดจ้าง
Prediction: จัดซื้อจัดจ้าง 



Compliant: นายสมชาย พงษ์ศิริ แจ้งพฤติกรรมเจ้าหน้าที่รัฐในจังหวัดปทุมธานีเรียกรับเงินใต้โต๊ะเพื่อเร่งรัดการจ่ายเงินค่าเวนคืนที่ดิน โดยมีหลักฐานเป็นคลิปเสียง
Target: เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์สินหรือประโยชน์อื่นใดแก่เจ้าหน้าที่ของรัฐ
Prediction: เรียกรับสินบน 



Compliant: นายสมชาย พงษ์ศิริ ขอให้ตรวจสอบกิจกรรมส่งมอบกระเป๋ายา อสม. ในเทศบาลนครอุดรธานี เนื่องจากมีข้อสงสัยเกี่ยวกับการจัดซื้ออุปกรณ์วิทยาศาสตร์การแพทย์ที่ไม่โปร่งใส โดยอ้างว่ามีการจัดซื้อจากบริษัท จีวาน โซลูชั่น จำกัด มูลค่า 7,318,925 บาท แต่ไม่มีการประกาศโครงการและข้อมูลการจัดซื้อในปี 2564-2565 ทำให้เกิดความสงสัยเกี่ยวกับการบริหารจัด